# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()

path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

user_log.createOrReplaceTempView("user_log")

# Question 1

Which page did user id ""(empty string) NOT visit?

In [2]:
spark.sql('''
SELECT DISTINCT(page)
FROM user_log
WHERE userId = ''
''').show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

# Question 3

How many female users do we have in the data set?

In [3]:
spark.sql('''
SELECT COUNT (DISTINCT userId)
FROM user_log
WHERE gender = 'F'
''').show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [4]:
spark.sql('''
SELECT artist, count(song) AS count
FROM user_log
GROUP BY artist
ORDER BY count DESC
LIMIT 1
''').show()

+--------+-----+
|  artist|count|
+--------+-----+
|Coldplay|   83|
+--------+-----+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [5]:
spark.udf.register('is_home', lambda x: int(x == 'Home'))

<function __main__.<lambda>(x)>

In [6]:
spark.sql('''
SELECT AVG(count) AS avg
FROM (
    SELECT userId, rank, count(*) AS count 
    FROM (
        SELECT userId, page, home,
        SUM(home) OVER (PARTITION BY userId ORDER BY ts DESC) AS rank
        FROM (
            SELECT userId, page, ts, is_home(page) AS home
            FROM user_log
            ) t
    ) t2
    WHERE page = 'NextSong'
    GROUP BY userId, rank
) t3
''').show()

+-----------------+
|              avg|
+-----------------+
|6.898347107438017|
+-----------------+

